In [ ]:
!pip install pandas
!pip install librosa
!pip install plotly
!pip install matplotlib
!pip install mutagen
!pip install pillow

In [ ]:
import tensorflow as tf

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import librosa
import os
from tqdm import tqdm

In [ ]:
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Dropout , Activation , Flatten
from tensorflow.keras.optimizers import Adam

In [ ]:
from sklearn import metrics 
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime

In [ ]:
import librosa
audio_file_path='/content/drive/MyDrive/02-Urban8k-grup6/audio/fold3/103199-4-2-8.wav'
librosa_audio_data,librosa_sample_rate=librosa.load(audio_file_path)

In [ ]:
print(librosa_audio_data)

In [ ]:
librosa_audio_data

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(librosa_audio_data)
plt.show()

In [ ]:
from IPython.display import Audio
Audio('/content/drive/MyDrive/02-Urban8k-grup6/audio/fold3/103199-4-2-8.wav')

In [ ]:
mfccs=librosa.feature.mfcc(y=librosa_audio_data, sr=librosa_sample_rate, n_mfcc=40)

In [ ]:
mfccs.shape

In [ ]:
audio_dataset_path='/content/drive/MyDrive/02-Urban8k-grup6/audio'
metadata=pd.read_csv('/content/drive/MyDrive/02-Urban8k-grup6/metadata/UrbanSound8K.csv')
metadata.head()

In [ ]:
def create_dataset_df(csv_file):
    dataset_df = pd.read_csv(csv_file)
    filepaths = []
    for i, row in dataset_df.iterrows():
        filepaths.append(os.path.join('/content/drive/MyDrive/02-Urban8k-grup6', 'fold'+str(row['fold']), row['slice_file_name']))
    dataset_df['filepath'] = filepaths
    return dataset_df

In [ ]:
def feature_extractor(csv_file):
    audio,sample_rate=librosa.load(file_name,res_type='kaiser_fast')
    mfccs_features=librosa.feature.mfcc(y=audio,sr=sample_rate,n_mfcc=40)
    mfcss_scaled_features=np.mean(mfccs_features.T,axis=0)

    return mfcss_scaled_features

In [ ]:
  extracted_features= []

for index_num,row in tqdm(metadata.iterrows()):
    file_name=os.path.join(os.path.abspath(audio_dataset_path),'/content/drive/MyDrive/02-Urban8k-grup6/audio', 'fold' +str(row['fold'])+'/',str(row['slice_file_name'])) 
    final_class_labels = row['class']   
    data=feature_extractor=(file_name)
    extracted_features.append([data,final_class_labels])

In [ ]:
extracted_features_df=pd.DataFrame(extracted_features,columns=["features","class"])
extracted_features_df.head()

In [ ]:
extracted_features_df.tail()

In [ ]:
x=np.array(extracted_features_df['features'].tolist())
y=np.array(extracted_features_df['class'].tolist())

In [ ]:
x.shape


In [ ]:
y.shape

In [ ]:
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))

In [ ]:
y

In [ ]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=0)

In [ ]:
xtrain

In [ ]:
num_labels=10

In [ ]:
model = Sequential()

model.add(Dense(125, input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(250))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(125))
model.add(Activation('relu'))
model.add(Dropout(0.5))


model.add(Dense(num_labels))
model.add(Activation('softmax'))


In [ ]:
model.summary()


In [ ]:
model.compile(loss="categorical_crossentropy",metrics=["accuracy"],optimizer="adam")


In [ ]:
num_epochs = 150
num_batch_size = 32

model.fit(
          xtrain, 
          ytrain, 
          batch_size=num_batch_size, 
          epochs=num_epochs,
          validation_data=(xtest, ytest),verbose=1
         )
# Maalesef bu noktada tıkandım ve daha ileriye gidemiyorum. Amacım oluşturduğum test oranınından yapılan tahminde 
# en yüksek orana sahip olanın görselle süslenip
# sesiyle beraber  ekrana yansıymasıydı. Urban8k yı Drive'a yükleme konusunda sıkıntı yaşadım çok uzun sürüyordu.
#Bir yerden drive paylaşımından kısayol ile kendi driveıma taşıdım. Ama üzerinde işlem yapamadığım için sıkıntı yaşamaktayım
#  ve maalesef kendim drive a kurucak kadar sürem de kalmadı çünkü discord üzerinden oluşturduğumuz grupta ki arkadaşlar hiç bir şekilde projeye katılmadılar.
# kendilerine projeye başlamamız gerektiği konusunda iletişimde bulunmama rağmen herhangi bir geri dönüş alamadım.bu yüzden herhangi bir destek alamadım ve projeyi kendi başıma
#yapmaya çalıştım.   
